In [ ]:
import os
import sys

# Import dependencies
import dolfin as dl
import numpy as np

sys.path.append(os.environ.get('HIPPYLIB_DIR', "../../"))

from hippylib import *

import logging

import matplotlib.pyplot as plt
% matplotlib inline

logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)
np.random.seed(seed=1)

In [ ]:
# Class HessianOperator to perform Hessian apply to a vector
class HessianOperator():
    cgiter = 0

    def __init__(self, R, Wmm, C, A, adj_A, W, Wum, bc0, use_gaussnewton=False):
        self.R = R
        self.Wmm = Wmm
        self.C = C
        self.A = A
        self.adj_A = adj_A
        self.W = W
        self.Wum = Wum
        self.bc0 = bc0
        self.use_gaussnewton = use_gaussnewton

        # incremental state
        self.du = dl.Vector()
        self.A.init_vector(self.du, 0)

        #incremental adjoint
        self.dp = dl.Vector()
        self.adj_A.init_vector(self.dp, 0)

        # auxiliary vector
        self.Wum_du = dl.Vector()
        self.Wum.init_vector(self.Wum_du, 1)

    def init_vector(self, v, dim):
        self.R.init_vector(v, dim)

    # Hessian performed on v, output as generic vector y
    def mult(self, v, y):
        self.cgiter += 1
        y.zero()
        if self.use_gaussnewton:
            self.mult_GaussNewton(v, y)
        else:
            self.mult_Newton(v, y)

    # define (Gauss-Newton) Hessian apply H * v
    def mult_GaussNewton(self, v, y):

        #incremental forward
        rhs = -(self.C * v)
        self.bc0.apply(rhs)
        dl.solve(self.A, self.du, rhs)

        #incremental adjoint
        rhs = - (self.W * self.du)
        self.bc0.apply(rhs)
        dl.solve(self.adj_A, self.dp, rhs)

        # Misfit term
        self.C.transpmult(self.dp, y)

        if self.R:
            Rv = self.R * v
            y.axpy(1, Rv)

    # define (Newton) Hessian apply H * v
    def mult_Newton(self, v, y):

        #incremental forward
        rhs = -(self.C * v)
        self.bc0.apply(rhs)
        dl.solve(self.A, self.du, rhs)

        #incremental adjoint
        rhs = -(self.W * self.du) - self.Wum * v
        self.bc0.apply(rhs)
        dl.solve(self.adj_A, self.dp, rhs)

        #Misfit term
        self.C.transpmult(self.dp, y)

        self.Wum.transpmult(self.du, self.Wum_du)
        y.axpy(1., self.Wum_du)

        y.axpy(1., self.Wmm * v)

        #Reg/Prior term
        if self.R:
            y.axpy(1., self.R * v)

In [ ]:
def id_str(nx, gamma, useTV, noise, useGaussNewton):
    return f'n={nx:d};gamma={gamma:.1e};{"TV" if useTV else "TN"};noise={noise:.2f};{"mixed" if callable(useGaussNewton) else "GN" if useGaussNewton else "N"}'


def AddDiffInverseProblem(nx, ny, gamma, v, morozov=False, plot=True, useTV=True, TVeps=0.1, m0_val=8, maxiter=50,
                          useGaussNewton=lambda iter, maxiter: iter < maxiter / 2, noise_level=0.01):
    mesh = dl.UnitSquareMesh(nx, ny)
    Vm = dl.FunctionSpace(mesh, 'Lagrange', 1)
    Vu = dl.FunctionSpace(mesh, 'Lagrange', 2)

    # The true and initial guess inverted parameter
    mtrue = dl.interpolate(dl.Expression('8. - 4.*(pow(x[0] - 0.5,2) + pow(x[1] - 0.5,2) < pow(0.2,2) )', degree=5), Vm)

    # define function for state and adjoint
    u = dl.Function(Vu)
    m = dl.Function(Vm)
    p = dl.Function(Vu)

    # define Trial and Test Functions
    u_trial, m_trial, p_trial = dl.TrialFunction(Vu), dl.TrialFunction(Vm), dl.TrialFunction(Vu)
    u_test, m_test, p_test = dl.TestFunction(Vu), dl.TestFunction(Vm), dl.TestFunction(Vu)

    # initialize input functions
    f = dl.interpolate(dl.Expression('std::max(0.5, exp(-25*(pow(x[0]-0.7, 2)+pow(x[1]-0.7, 2))))', degree=5), Vu)
    k = dl.Constant(1.0)
    u0 = dl.Constant(0.0)

    # set up dirichlet boundary conditions
    def boundary(x, on_boundary):
        return on_boundary

    bc_state = dl.DirichletBC(Vu, u0, boundary)
    bc_adj = dl.DirichletBC(Vu, dl.Constant(0.), boundary)

    # weak form for setting up the synthetic observations
    def a_state_gen(u, m, p_hat):
        return (dl.inner(k * dl.grad(u), dl.grad(p_hat)) * dl.dx
                + dl.inner(v, dl.grad(u)) * p_hat * dl.dx
                + 100 * dl.exp(m) * u ** 3 * p_hat * dl.dx
                - f * p_hat * dl.dx)

    a_true_not_used = (dl.inner(k * dl.grad(u_trial), dl.grad(u_test)) * dl.dx
                       + dl.inner(v, dl.grad(u_trial)) * u_test * dl.dx)
    L_true_not_used = f * u_test * dl.dx

    def forward_solver(_u, m):
        dl.solve(a_state_gen(_u, m, u_test) == 0, _u, bc_state,
                 solver_parameters={'newton_solver': {'relative_tolerance': 1e-5, 'maximum_iterations': 200}})

    # solve the forward/state problem to generate synthetic observations
    A_true_not_used, b_true_not_used = dl.assemble_system(a_true_not_used, L_true_not_used, bc_state)

    utrue = dl.Function(Vu)
    # dl.solve(A_true_not_used, utrue.vector(), b_true_not_used)
    forward_solver(utrue, mtrue)

    ud = dl.Function(Vu)
    ud.assign(utrue)

    # perturb state solution and create synthetic measurements ud
    # ud = u + ||u||/SNR * random.normal
    MAX = ud.vector().norm("linf")
    noise = dl.Vector()
    A_true_not_used.init_vector(noise, 1)
    noise.set_local(noise_level * MAX * np.random.normal(0, 1, len(ud.vector().get_local())))
    bc_adj.apply(noise)

    ud.vector().axpy(1., noise)

    # define cost function
    def cost(u, ud, m, gamma):
        if useTV:
            g = dl.grad(m)
            reg = gamma * dl.assemble(dl.sqrt(dl.inner(g, g) + TVeps) * dl.dx)
        else:
            reg = 0.5 * gamma * dl.assemble(dl.inner(dl.grad(m), dl.grad(m)) * dl.dx)
        misfit = 0.5 * dl.assemble((u - ud) ** 2 * dl.dx)

        return [reg + misfit, misfit, reg]

    # weak form for setting up the state equation
    # a_state = (dl.inner(k * dl.grad(u_trial), dl.grad(u_test)) * dl.dx
    #           + dl.dot(v, dl.grad(u_trial)) * u_test * dl.dx
    #           + 100 * dl.exp(m) * u_trial ** 3 * u_test * dl.dx)
    # L_state = f * u_test * dl.dx

    # weak form for setting up the adjoint equation
    a_adj = (dl.inner(k * dl.grad(p_trial), dl.grad(p_test)) * dl.dx
             + dl.dot(v, dl.grad(p_test)) * p_trial * dl.dx
             + 300 * dl.exp(m) * u ** 2 * p_test * p_trial * dl.dx)
    L_adj = -dl.inner(u - ud, p_test) * dl.dx

    # weak form for setting up the increment state equation
    a_inc_state = (dl.inner(k * dl.grad(u_trial), dl.grad(u_test)) * dl.dx
                   + dl.dot(v, dl.grad(u_trial)) * u_test * dl.dx
                   + 300 * dl.exp(m) * u ** 3 * u_trial * u_test * dl.dx)
    L_inc_state_not_used = u_test * dl.dx

    # weak form for gradient
    CTvarf = 100 * dl.exp(m) * m_test * u ** 3 * p * dl.dx
    if useTV:
        gradRvarf = ((gamma / dl.sqrt(dl.inner(dl.grad(m), dl.grad(m)) + TVeps)) *
                     dl.inner(dl.grad(m), dl.grad(m_test)) * dl.dx)
    else:
        gradRvarf = gamma * dl.inner(dl.grad(m), dl.grad(m_test)) * dl.dx

    # L^2 weighted inner product
    M_varf = dl.inner(m_trial, m_test) * dl.dx
    M = dl.assemble(M_varf)

    m0 = dl.interpolate(dl.Constant(m0_val), Vm)
    m.assign(m0)

    # solve state equation
    # state_A, state_b = dl.assemble_system(a_state, L_state, bc_state)
    # dl.solve (state_A, u.vector(), state_b)
    forward_solver(u, m)

    nb.multi1_plot([ud, u], ['ud', 'u'])
    plt.show()

    # evaluate cost
    [cost_old, misfit_old, reg_old] = cost(u, ud, m, gamma)

    #Hessian varfs
    W_varf = dl.inner(u_trial, u_test) * dl.dx
    if useTV:
        _g = dl.grad(m)
        _g2 = dl.inner(_g, _g)
        _g_trial = dl.grad(m_trial)
        _g_test = dl.grad(m_test)
        R_varf = gamma / dl.sqrt(_g2 + TVeps) ** 3 * (
                dl.inner(_g_trial, _g_test) * (_g2 + TVeps)
                - dl.inner(_g, _g_trial) * dl.inner(_g, _g_test)
        ) * dl.dx
    else:
        R_varf = dl.Constant(gamma) * dl.inner(dl.grad(m_trial), dl.grad(m_test)) * dl.dx

    C_varf = 100 * dl.exp(m) * m_trial * u ** 3 * u_test * dl.dx
    Wum_varf = 300 * dl.exp(m) * m_trial * u ** 2 * p_test * p * dl.dx
    Wmm_varf = 100 * dl.exp(m) * m_trial * m_test * u ** 3 * p * dl.dx

    # Assemble constant matrices
    W = dl.assemble(W_varf)
    R = dl.assemble(R_varf)

    # define parameters for the optimization
    tol = 1e-8
    c = 1e-4

    # initialize iter counters
    iter = 1
    total_cg_iter = 0
    converged = False

    # initializations
    g, m_delta = dl.Vector(), dl.Vector()
    R.init_vector(m_delta, 0)
    R.init_vector(g, 0)

    m_prev = dl.Function(Vm)

    id = id_str(nx, gamma, useTV, noise_level, useGaussNewton)
    import pathlib
    pathlib.Path(f'/home/fenics/shared/hw5/plot/{id}').mkdir(parents=True, exist_ok=True)
    pathlib.Path(f'/home/fenics/shared/hw5/data/{id}').mkdir(parents=True, exist_ok=True)

    checkpoints = []

    print("Nit   CGit   cost          misfit        reg           sqrt(-G*D)    ||grad||       alpha  tolcg")

    while iter < maxiter and not converged:

        # solve the adoint problem
        adjoint_A, adjoint_RHS = dl.assemble_system(a_adj, L_adj, bc_adj)
        dl.solve(adjoint_A, p.vector(), adjoint_RHS)

        # evaluate the  gradient
        MG = dl.assemble(CTvarf + gradRvarf)

        # calculate the L^2 norm of the gradient
        dl.solve(M, g, MG)
        grad2 = g.inner(MG)
        gradnorm = np.sqrt(grad2)


        # set the CG tolerance (use Eisenstat–Walker termination criterion)
        if iter == 1:
            gradnorm_ini = gradnorm
        tolcg = min(0.5, np.sqrt(gradnorm / gradnorm_ini))

        # assemble W_um and W_mm
        C = dl.assemble(C_varf)
        Wum = dl.assemble(Wum_varf)
        Wmm = dl.assemble(Wmm_varf)

        # define the Hessian apply operator (with preconditioner)
        inc_state_A, _ = dl.assemble_system(a_inc_state, L_inc_state_not_used, bc_state)
        R = dl.assemble(R_varf)
        Hess_Apply = HessianOperator(R, Wmm, C, inc_state_A, adjoint_A, W, Wum, bc_adj,
                                     use_gaussnewton=useGaussNewton(iter, maxiter) if callable(useGaussNewton) else useGaussNewton)
        P = R + 0.1 * gamma * M
        Psolver = dl.PETScKrylovSolver("cg", amg_method())
        Psolver.set_operator(P)

        solver = CGSolverSteihaug()
        solver.set_operator(Hess_Apply)
        solver.set_preconditioner(Psolver)
        solver.parameters["rel_tolerance"] = tolcg
        solver.parameters["zero_initial_guess"] = True
        solver.parameters["print_level"] = -1
        solver.parameters["max_iter"] = 30

        # solve the Newton system H a_delta = - MG
        solver.solve(m_delta, -MG)
        total_cg_iter += Hess_Apply.cgiter

        # if plot and iter % 10 == 0:
        #     g_form = dl.Function(Vm)
        #     g_form.vector()[:] = g
        #     m_delta_form = dl.Function(Vm)
        #     m_delta_form.vector()[:] = m_delta
        #     nb.multi1_plot([p, g_form], ['p', 'g'], same_colorbar=False)
        #     nb.multi1_plot([m_delta_form, m], ['m_delta', 'm'], same_colorbar=False)
        #     plt.show()

        # linesearch
        alpha = 1
        descent = 0
        no_backtrack = 0
        m_prev.assign(m)
        while descent == 0 and no_backtrack < 10:
            m.vector().axpy(alpha, m_delta)

            # solve the state/forward problem
            # state_A, state_b = dl.assemble_system(a_state, L_state, bc_state)
            # dl.solve(state_A, u.vector(), state_b)
            forward_solver(u, m)

            # evaluate cost
            [cost_new, misfit_new, reg_new] = cost(u, ud, m, gamma)

            # check if Armijo conditions are satisfied
            if cost_new < cost_old + alpha * c * MG.inner(m_delta):
                cost_old = cost_new
                descent = 1
            else:
                no_backtrack += 1
                alpha *= 0.5
                m.assign(m_prev)  # reset a

        # calculate sqrt(-G * D)
        graddir = np.sqrt(- MG.inner(m_delta))

        sp = ""
        print("%2d %2s %2d %3s %8.5e %1s %8.5e %1s %8.5e %1s %8.5e %1s %8.5e %1s %5.2f %1s %5.3e" %
              (iter, sp, Hess_Apply.cgiter, sp, cost_new, sp, misfit_new, sp, reg_new, sp,
               graddir, sp, gradnorm, sp, alpha, sp, tolcg))

        # check for convergence
        if gradnorm < tol and iter > 1:
            converged = True
            print("Newton's method converged in ", iter, "  iterations")
            print("Total number of CG iterations: ", total_cg_iter)

        iter += 1

        checkpoints.append({'u': u.vector().get_local(),
                            'm': m.vector().get_local(),
                            'p': p.vector().get_local()})

    if not converged:
        print("Newton's method did not converge in ", maxiter, " iterations")

    if plot:
        nb.multi1_plot([mtrue, m], ["mtrue", "m"])
        plt.show()

    np.savez(f'/home/fenics/shared/hw5/data/{id}/data.npz', checkpoints=checkpoints, m0_val=m0_val,
             gamma=gamma, useTV=useTV, mtrue=mtrue.vector().get_local(), iter=iter, total_cg_iter=total_cg_iter)

    Mstate = dl.assemble(u_trial * u_test * dl.dx)
    noise_norm2 = noise.inner(Mstate * noise)
    if not morozov:
        Hmisfit = HessianOperator(None, Wmm, C, inc_state_A, adjoint_A, W, Wum, bc_adj, use_gaussnewton=False)
        k = 50
        p = 20

        Omega = MultiVector(m.vector(), k + p)
        parRandom.normal(1., Omega)
        lmbda, evecs = doublePassG(Hmisfit, P, Psolver, Omega, k)

        plt.plot(range(0, k), lmbda, 'b*', range(0, k + 1), np.ones(k + 1), '-r')
        plt.yscale('log')
        plt.xlabel('number')
        plt.ylabel('eigenvalue')
        plt.show()

        nb.plot_eigenvectors(Vm, evecs, mytitle="Eigenvector", which=[0, 1, 2, 5, 10, 15])
        plt.savefig(f'/home/fenics/shared/hw5/plot/{id}/eigenvectors.jpg')
        plt.show()

    return Vm.dim(), iter, total_cg_iter, noise_norm2, cost_new, misfit_new, reg_new

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-9, morozov=False,
                          plot=True, useTV=False, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-7, morozov=True,
                          plot=True, useTV=False, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-8, morozov=True,
                          plot=True, useTV=False, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-10, morozov=True,
                          plot=True, useTV=False, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-9, morozov=False,
                          plot=True, useTV=True, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-7, morozov=True,
                          plot=True, useTV=True, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-8, morozov=True,
                          plot=True, useTV=True, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
    AddDiffInverseProblem(nx=20, ny=20, v=dl.Constant((1., 0.)), gamma=1e-10, morozov=True,
                          plot=True, useTV=True, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
for nx in 40, 80:
    for useTV in True, False:
        for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
            ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
                AddDiffInverseProblem(nx=nx, ny=nx, v=dl.Constant((1., 0.)), gamma=gamma, morozov=True,
                                      plot=True, useTV=useTV, TVeps=0.001, m0_val=8, useGaussNewton=True))

In [ ]:
# Read in optimized data and plot them
m_true_dict = dict()
m_recovered_dict = dict()
iters_dict = dict()
cg_iters_dict = dict()

for useTV in True, False:
    m_recovered_dict[useTV] = dict()
    iters_dict[useTV] = dict()
    cg_iters_dict[useTV] = dict()
    for nx in 20, 40, 80:
        m_true_dict[nx] = dict()
        m_recovered_dict[useTV][nx] = dict()
        iters_dict[useTV][nx] = dict()
        cg_iters_dict[useTV][nx] = dict()

        mesh = dl.UnitSquareMesh(nx, nx)
        Vm = dl.FunctionSpace(mesh, 'Lagrange', 1)
        for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
            id = id_str(nx, gamma, useTV, 0.01, True)
            with np.load(f'/home/fenics/shared/hw5/data/{id}/data.npz') as data:
                checkpoint_vec = data['checkpoints']

                m_recovered = dl.Function(Vm)
                m_recovered.vector()[:] = checkpoint_vec[-1]['m']

                m_true = dl.Function(Vm)
                m_true.vector()[:] = data['mtrue']

                m_true_dict[nx] = m_true
                m_recovered_dict[useTV][nx][gamma] = m_recovered
                iters_dict[useTV][nx][gamma] = data['iter']
                cg_iters_dict[useTV][nx][gamma] = data['total_cg_iter']

In [ ]:
for nx in 20, 40, 80:
    nb.plot(m_true_dict[nx], vmin=4, vmax=8)
    plt.title('$m_{true}' + f',n={nx:d}$')
    plt.savefig(f'/home/fenics/shared/hw5/plot/m_true_n_{nx:d}.jpg', bbox_inches='tight', transparent="True", pad_inches=0)
    plt.show()

for useTV in True, False:
    for nx in 20, 40, 80:
        for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
            id = id_str(nx, gamma, useTV, 0.01, True)

            m_recovered = m_recovered_dict[useTV][nx][gamma]
            iter = iters_dict[useTV][nx][gamma]
            cg_iter = cg_iters_dict[useTV][nx][gamma]
            nb.plot(m_recovered, vmin=4, vmax=8, colorbar=False)
            plt.title(f'$\\beta$={gamma:.0e}, {iter}/{cg_iter} iters')
            plt.savefig(f'/home/fenics/shared/hw5/plot/{id}/m_recovered.jpg', bbox_inches='tight', transparent="True", pad_inches=0)
            plt.show()

In [ ]:
nx = 40
useTV = False
for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
    for noise_level in 0.01, 0.1:
        for useGaussNewton in False, True:
            ndof, niter, ncgiter, noise_norm2, cost, misfit, reg = (
                AddDiffInverseProblem(nx=nx, ny=nx, v=dl.Constant((1., 0.)), gamma=gamma, morozov=True,
                                      plot=True, useTV=useTV, m0_val=8, useGaussNewton=useGaussNewton,
                                      noise_level=noise_level, maxiter=100))

In [ ]:
# Read in optimized data and plot them
nx = 40
useTV = False
mesh = dl.UnitSquareMesh(nx, nx)
Vm = dl.FunctionSpace(mesh, 'Lagrange', 1)

m_recovered_dict_2 = dict()
iters_dict_2 = dict()
cg_iters_dict_2 = dict()

for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
    m_recovered_dict_2[gamma] = dict()
    iters_dict_2[gamma] = dict()
    cg_iters_dict_2[gamma] = dict()

    for noise_level in 0.01, 0.1:
        m_recovered_dict_2[gamma][noise_level] = dict()
        iters_dict_2[gamma][noise_level] = dict()
        cg_iters_dict_2[gamma][noise_level] = dict()

        for useGaussNewton in False, True:
            id = id_str(nx, gamma, useTV, noise_level, useGaussNewton)
            with np.load(f'/home/fenics/shared/hw5/data/{id}/data.npz') as data:
                checkpoint_vec = data['checkpoints']

                m_recovered = dl.Function(Vm)
                m_recovered.vector()[:] = checkpoint_vec[-1]['m']

                m_true = dl.Function(Vm)
                m_true.vector()[:] = data['mtrue']

                m_recovered_dict_2[gamma][noise_level][useGaussNewton] = m_recovered
                iters_dict_2[gamma][noise_level][useGaussNewton] = data['iter']
                cg_iters_dict_2[gamma][noise_level][useGaussNewton] = data['total_cg_iter']

In [ ]:
for gamma in 1e-10, 1e-9, 1e-8, 1e-7:
    for noise_level in 0.01, 0.1:
        for useGaussNewton in False, True:
            id = id_str(nx, gamma, useTV, noise_level, useGaussNewton)

            m_recovered = m_recovered_dict_2[gamma][noise_level][useGaussNewton]
            iter = iters_dict_2[gamma][noise_level][useGaussNewton]
            cg_iter = cg_iters_dict_2[gamma][noise_level][useGaussNewton]
            nb.plot(m_recovered, vmin=4, vmax=8, colorbar=False)
            plt.title(f'$\\beta$={gamma:.0e}, {iter}/{cg_iter} iters')
            plt.savefig(f'/home/fenics/shared/hw5/plot/{id}/m_recovered.jpg', bbox_inches='tight', transparent="True", pad_inches=0)
            plt.show()

In [ ]:
gammas = np.array([1e-10, 1e-9, 1e-8, 1e-7])
def plot_wrapper(ax, iter_dict, noise_level, useGaussNewton, style):
    iters = np.array([iter_dict[gamma][noise_level][useGaussNewton] for gamma in gammas])
    ax.plot(gammas, iters, style, label=f'{int(noise_level*100):d}% noise, {"Gauss-Newton" if useGaussNewton else "Newton"}')

fig, axs = plt.subplots(2, 1, figsize=(5, 5), squeeze=True)
for noise_level, color in (0.01, 'r'), (0.1, 'b'):
    for useGaussNewton, style in (True, '-'), (False, '--'):
        plot_wrapper(axs[0], iters_dict_2, noise_level, useGaussNewton, color+style)
        plot_wrapper(axs[1], cg_iters_dict_2, noise_level, useGaussNewton, color+style)

for ax in axs:
    ax.set(xscale='log', xlabel='$\\beta$', ylabel='# iteration')
    ax.legend(loc='best')
    ax.grid(True)

plt.savefig(f'/home/fenics/shared/hw5/plot/itr.jpg')
plt.show()